In [2]:
print("En esta ocasión desarrollaremos un agente que ayude en la gestión efectiva del tiempo \
      \n el mismo, medirá la “productividad” del usuario. Dicha “productividad” corresponde \
      \n a la cantidad de tareas realizadas en un tiempo determinado. \
      \n A su vez, el agente le recomendara al usuario acciones a tomar para un mayor provecho del tiempo.")

En esta ocasión desarrollaremos un agente que ayude en la gestión efectiva del tiempo       
 el mismo, medirá la “productividad” del usuario. Dicha “productividad” corresponde       
 a la cantidad de tareas realizadas en un tiempo determinado.       
 A su vez, el agente le recomendara al usuario acciones a tomar para un mayor provecho del tiempo.


In [3]:
print("Como libería, pensamos utilizar MESA de python puesto que en la descripción de la librería \
      \n nos indica que está orientado a horarios de agentes, por lo cual podríamos basarnos \
      \n en algunos de los métodos que ya la librería implementa,  \
      \n sin embargo, es posible que también utilicemos otras liberías como AgentPy \
      \n puesto que en Mesa no estará todo lo que necesitamos ")

Como libería, pensamos utilizar MESA de python puesto que en la descripción de la librería       
 nos indica que está orientado a horarios de agentes, por lo cual podríamos basarnos       
 en algunos de los métodos que ya la librería implementa,        
 sin embargo, es posible que también utilicemos otras liberías como AgentPy       
 puesto que en Mesa no estará todo lo que necesitamos 


In [14]:
print("Estas son las importaciones de las librerías orientadas a IA que utilizaremos:")
print("mesa, agentpy, ipysimulate")
print("Usaremos además otras librerías auxiliares, cómo:")
print("numpy, pandas, anytree")

import numpy
import mesa
import agentpy
import ipysimulate
import ipywidgets
import pandas as pd
from anytree import NodeMixin, RenderTree

Estas son las importaciones de las librerías orientadas a IA que utilizaremos:
mesa, agentpy, ipysimulate
Usaremos además otras librerías auxiliares, cómo:
numpy, pandas, anytree


In [5]:
#definamos la estructura más básica de nuestro agente
#la cual podrá ir creciendo según se definan nuevas características

class ScheduleAgent:
  def __init__(self, name, age, gender, dailyTask, comments):
    self.name = name
    self.age = age
    self.gender = gender
    self.dailyTask = dailyTask
    self.notification = comments
    self.comments = comments

taskList = pd.read_csv('taskList.csv')
comentarios = ""

my_Agent = ScheduleAgent("Pepito", "20", "Masculino", taskList, comentarios)

In [6]:
# Estructura del árbol con el uso de la librería anytree
# pip install anytree

class Task:
    def __init__(self, id, name, avgTime, urgency):
        self.id=id
        self.name=name
        self.avgTime=avgTime
        self.urgency=urgency
    
    def __str__(self):
        return f"Name: {self.name}, AvgTime: {self.avgTime}, Urgency: {self.urgency}"

class TaskClass(Task, NodeMixin):  # Add Node feature
    def __init__(self, name, avgTime, urgency, parent=None, children=None):
        super(Task, self).__init__()
        self.name = name
        self.avgTime = avgTime
        self.urgency = urgency
        self.parent = parent
        if children:
            self.children = children

In [7]:
def getNodesList(taskList:pd.DataFrame) -> list:
    nodeList = []
    for i in range(0, len(taskList["Name"])):
        nodeList.append(TaskClass(
            name = taskList["Name"][i],
            avgTime = taskList["AvgTime"][i],
            urgency = taskList["Urgency"][i]
        ))
    return nodeList



def setParentNodes(taskList:pd.DataFrame, nodeList:list) -> TaskClass:
    for i in range(1, len(nodeList)):
        node = nodeList[i]
        node.parent = nodeList[(int(taskList["Parent"][i]))-1]
    return nodeList[0] #retorno root del arbol ya que desde root lo puedo recorrer todo



def buildTree(taskList:pd.DataFrame) -> TaskClass: #construccion del arbol por medio de archivo CSV
    nodeList = getNodesList(taskList)
    rootNode = setParentNodes(taskList, nodeList)
    return rootNode

In [8]:
def printTree(rootNode: TaskClass) -> None:
    for pre, fill, node in RenderTree(rootNode):
        print("%s%s" % (pre, node.name))    

In [9]:
task_df = pd.read_csv('taskList.csv') 
task_tree = buildTree(task_df)

# print(task_df)
printTree(task_tree)

desayunar
├── bañarse
│   ├── hacer la tarea
│   │   ├── almorzar
│   │   └── hacer ejercicio
│   └── estudiar para el parcial
│       ├── escuchar musica
│       └── leer documentacion
└── sacar al perro
    ├── hacer estiramientos
    │   ├── ver netflix
    │   └── reunirse con amigos
    └── regar plantas
        ├── ordenar el cuarto
        └── jugar con el perro


In [10]:
# Diferentes formas de imprimir nuestros datos con anytree
print("task tree: " + str(task_tree))
print("task tree.name: " + task_tree.name)
print("task tree.children: " + str(task_tree.children))
print("task tree.children left: " + str(task_tree.children[0])) #IZQ
print("task tree.children right: " + str(task_tree.children[1])) #DER
print(type(task_tree.children))
print("task tree.height: " + str(task_tree.height))
print("task tree.root: " + str(task_tree.root))
print("task tree.children left name: " + str(task_tree.children[0].name)) #IZQ
print("task tree.children left name: " + str(task_tree.children[0].children[0])) #IZQ

task tree: Name: desayunar, AvgTime: 40, Urgency: 5
task tree.name: desayunar
task tree.children: (<__main__.TaskClass object at 0x00000131B5DD2A50>, <__main__.TaskClass object at 0x00000131B5DD2E50>)
task tree.children left: Name: bañarse, AvgTime: 45, Urgency: 2
task tree.children right: Name: sacar al perro, AvgTime: 30, Urgency: 5
<class 'tuple'>
task tree.height: 3
task tree.root: Name: desayunar, AvgTime: 40, Urgency: 5
task tree.children left name: bañarse
task tree.children left name: Name: hacer la tarea, AvgTime: 60, Urgency: 7


In [13]:
# Valores iniciales
current_node = task_tree
is_max = True

class Choice():
    def __init__(self, move, urgency):
        self.move = move
        self.urgency = urgency

    def __str__(self):
        return self.move + ": " + str(self.urgency)


# Algoritmo minimax
def miniMax(rootNode, isMax):
    children = rootNode.children
    try:
        l_choice = miniMax(children[0], not isMax)
        r_choice = miniMax(children[1], not isMax)

        if (isMax):
            if (l_choice.urgency > r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency)
        else:
            if (l_choice.urgency < r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency) 
    except IndexError:
        return Choice("end", rootNode.urgency)

# Ciclo principal de ejecución
while (True):
    # run minimax on current node
    choice = miniMax(current_node, is_max)
    
    # make choice based on minimax search
    if (choice.move == "left"):
        print ("Nos movemos a la izquierda con una urgencia de: " + str(current_node.children[0].urgency))
        current_node = current_node.children[0]
    elif (choice.move == "right"):
        print ("Nos movemos a la derecha con una urgencia de: " + str(current_node.children[1].urgency))
        current_node = current_node.children[1]
    elif (choice.move == "end"):
        print ("La urgencia del último nodo hoja es: " + str(choice.urgency))
        break
    

Nos movemos a la derecha con una urgencia de: 5
Nos movemos a la derecha con una urgencia de: 7
Nos movemos a la derecha con una urgencia de: 6
La urgencia del último nodo hoja es: 6


In [ ]:
rootNode = buildTree(my_Agent.dailyTask) #contruccion del arbol
i = 10
while (i < 10):
    choice = miniMax(rootNode, True)
print(choice)


right: 4
